# MonaLog Helper

## Familialize CCG loading

In [1]:
import sys

In [2]:
sys.path.append('src')

In [3]:
import sick_ans

In [4]:
from getMono import CCGtrees, CCGtree, ErrorCCGtree, ErrorCompareSemCat, eprint

In [5]:
trees = CCGtrees(fn_log="src/sick_uniq.raw.tok.preprocess.log")


reading log file: src/sick_uniq.raw.tok.preprocess.log
reading log file done!



In [6]:
trees.readEasyccgStr("src/sick_uniq.raw.depccg.parsed.txt")

reading trees from easyccg / depccg output ...

trees read in from easyccg / depccg output!




In [7]:
sick2uniq = sick_ans.sick2uniq
sick2uniq[4]

{'P': 5047, 'H': 5117}

In [8]:
P = trees.build_one_tree(sick2uniq[4]['P'], "easyccg")
H = trees.build_one_tree(sick2uniq[4]['H'], "easyccg")

building tree 5047...
building tree 5117...


In [9]:
P

a *None* young *None* boy *None* be *None* play *None* outdoors *None* and *None* a *None* man *None* be *None* smile *None* nearby *None*

In [10]:
vars(P.leafNodes[0])

{'parent': nt: NP ((e,t),t) fa 2 None False,
 'children': [],
 'sisters': [nt: N (e,t) fa 3 None False],
 'depth': 3,
 'cat': NP/N,
 'chunk': 'XX',
 'entity': 'O',
 'lemma': 'a',
 'pos': 'DT',
 'span': 1,
 'start': 0,
 'word': 'a',
 'wholeStr': 'A',
 'word_raw': 'A',
 'visited': True,
 'span_id': None,
 'impType': <getMono.ImpType at 0x1119ee550>,
 'impSign': None,
 'fixed': False,
 'note': None,
 'number': None}

In [11]:
for n in P.nonTermNodes:
    print(n.wholeStr)

YOUNG BOY
A YOUNG BOY
PLAY OUTDOORS
BE PLAY OUTDOORS
A YOUNG BOY BE PLAY OUTDOORS
A MAN
SMILE NEARBY
BE SMILE NEARBY
A MAN BE SMILE NEARBY
AND A MAN BE SMILE NEARBY


In [12]:
vars(P.nonTermNodes[-1])

{'parent': nt: S[dcl] t ba 0 None False,
 'children': [lf: conj CONJ and CC 2 mono:None imp:None True,
  nt: S[dcl] t ba 2 None False],
 'sisters': [nt: S[dcl] t ba 1 None False],
 'depth': 1,
 'cat': S[dcl]\S[dcl],
 'ruleType': 'conj',
 'wholeStr': 'AND A MAN BE SMILE NEARBY',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x1118bfb80>,
 'impSign': None,
 'note': None,
 'number': None}

In [13]:
P.root.wholeStr

'A YOUNG BOY BE PLAY OUTDOORS AND A MAN BE SMILE NEARBY'

In [14]:
P.root.ruleType

'ba'

In [15]:
P.root.children

[nt: S[dcl] t ba 1 None False, nt: S[dcl]\S[dcl] (t,t) conj 1 None False]

# Conjunction Related Errors (Subsequence Heuristic)

### Separating A Conjucted Sentence into two sentences

#### Familializing with CCG tree

In [16]:
CONJ_ID = [5003,987,6287,7311,6544,9571,7311,9136]

In [17]:
P = trees.build_one_tree(sick2uniq[5003]['P'], "easyccg")
H = trees.build_one_tree(sick2uniq[5003]['H'], "easyccg")

building tree 71...
building tree 455...


In [18]:
P

a *None* big *None* cat *None* be *None* open *None* a *None* plastic *None* drawer *None* with *None* its *None* paw *None* and *None* be *None* jump *None* inside *None*

In [19]:
H.wholeStr

'A CAT BE OPEN A DRAWER AND JUMP INSIDE'

In [20]:
vars(P.root.children[1])

{'parent': nt: S[dcl] t ba 0 None False,
 'children': [nt: S[dcl]\NP (((e,t),t),t) fa 2 None False,
  nt: (S[dcl]\NP)\(S[dcl]\NP) ((((e,t),t),t),(((e,t),t),t)) conj 2 None False],
 'sisters': [nt: NP ((e,t),t) fa 1 None False],
 'depth': 1,
 'cat': S[dcl]\NP,
 'ruleType': 'ba',
 'wholeStr': 'BE OPEN A PLASTIC DRAWER WITH ITS PAW AND BE JUMP INSIDE',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x111874fd0>,
 'impSign': None,
 'note': None,
 'number': None}

In [21]:
def checkSentConj(P):
    '''Checks if P has sentential or verbal conjunction'''
    res = False
    root = P.root
    dep = 0
    while len(root.children[1].children) != 0:
        dep += 1
        Lchild = root.children[1]
#         print(Lchild.wholeStr)
#         print(vars(Lchild))
        if Lchild.ruleType == 'conj':
            res = True
            break
        root = Lchild
    return res, dep

In [22]:
# testing checkSentConj
for i in CONJ_ID:
    P = trees.build_one_tree(sick2uniq[i]['P'], "easyccg")
    print(i, checkSentConj(P))
    print()

5003 (True, 2)

987 (True, 2)

6287 (True, 1)

7311 (False, 5)

6544 (True, 2)

9571 (True, 2)

7311 (False, 5)

9136 (False, 4)



building tree 71...
building tree 3315...
building tree 167...
building tree 549...
building tree 4304...
building tree 3344...
building tree 549...
building tree 1413...


### Sentence extraction (from conjuncted sentences): NP VP1 and VP2 -> NP VP1 / NP VP2

In [23]:
def ConjEntail(P):
    '''Returning sentences'''
    k = []
    res, dep = checkSentConj(P)
    if res and dep == 2:
        ### Case1: Conjunciton of VP
        root = P.root
        Subj = root.children[0]
        ConjVP = root.children[1]
        print("Subj: {}".format(Subj.wholeStr))
        print("ConjVP: {}".format(ConjVP.wholeStr))
        
        while ConjVP.children[1].ruleType == 'conj':
            tmp = Subj.wholeStr + " " + ConjVP.children[0].wholeStr
            k.append(tmp)
            ConjVP = ConjVP.children[1]
        tmp = Subj.wholeStr + " " + ConjVP.children[1].wholeStr
        k.append(tmp)
    return k

In [24]:
def ConjEntail(P):
    '''Returning trees'''
    k = []
    res, dep = checkSentConj(P)
    if res and dep == 2:
        ### Case1: Conjunciton of VP
        root = P.root
        Subj = root.children[0]
        ConjVP = root.children[1]
        print("Subj: {}".format(Subj.wholeStr))
        print("ConjVP: {}".format(ConjVP.wholeStr))
        
        # iteratively add sentences
        # eg ConjVP = VP1 and VP2
        # then generate Subj VP1 and Subj VP2
        while ConjVP.children[1].ruleType == 'conj':
            tmp = root.copy()
            tmp.children[1] = ConjVP.children[0]
            tmp.wholeStr = Subj.wholeStr + " " + ConjVP.children[0].wholeStr
            k.append(tmp)
            ConjVP = ConjVP.children[1]
        tmp = root.copy()
        tmp.children[1] = ConjVP.children[1]
        tmp.wholeStr = Subj.wholeStr + " " + ConjVP.children[1].wholeStr
        k.append(tmp)
    return k

In [25]:
k = ConjEntail(P)

In [26]:
k

[]

In [27]:
for t in k:
    print(t.wholeStr)
    for c in t.children:
        print(vars(c))
        print()
    print()

### Testing ConjEntail

In [28]:
for i in CONJ_ID:
    P = trees.build_one_tree(sick2uniq[i]['P'], "easyccg")
    H = trees.build_one_tree(sick2uniq[i]['H'], "easyccg")
    k = ConjEntail(P)
    k_str = []
    for t in k:
        k_str.append(t.wholeStr)
        
    print(i, P.wholeStr)
    print(H.wholeStr)
    print(k_str)
    print(H.wholeStr in k_str)
    print()

building tree 71...
building tree 455...
building tree 3315...
building tree 5089...
building tree 167...
building tree 166...
building tree 549...
building tree 2070...
building tree 4304...
building tree 5228...


Subj: A BIG CAT
ConjVP: BE OPEN A PLASTIC DRAWER WITH ITS PAW AND BE JUMP INSIDE
5003 A BIG CAT BE OPEN A PLASTIC DRAWER WITH ITS PAW AND BE JUMP INSIDE
A CAT BE OPEN A DRAWER AND JUMP INSIDE
['A BIG CAT BE OPEN A PLASTIC DRAWER WITH ITS PAW', 'A BIG CAT BE JUMP INSIDE']
False

Subj: AN ADULT
ConjVP: BE IN A AMPHITHEATER AND BE TALK TO A BOY
987 AN ADULT BE IN A AMPHITHEATER AND BE TALK TO A BOY
NO ADULT IN A AMPHITHEATER BE TALK TO A BOY
['AN ADULT BE IN A AMPHITHEATER', 'AN ADULT BE TALK TO A BOY']
False

6287 A BLACK DOG BE RUN IN A GRASS AND ITS TONGUE BE HANG OUT
A BLACK DOG BE RUN IN A GRASS
[]
False

7311 A MAN BE SMOKE A CIGARETTE IN A STRAW HAT
A MAN BE WEAR A STRAW HAT AND SMOKE A CIGARETTE
[]
False

Subj: A ICE HOCKEY GOALKEEPER
ConjVP: BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL
6544 A ICE HOCKEY GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL
NO HOCKEY PLAYER IN A YELLOW JERSEY BE GUARD A GOAL
['A ICE HOCKEY GOALKEEPER BE WEAR A YELLOW JERSEY', 'A ICE HOCKEY GOALK

building tree 3344...
building tree 3345...
building tree 549...
building tree 2070...
building tree 1413...
building tree 1990...


## Specific to General: Adj N VP -> N VP

In [29]:
P = trees.build_one_tree(sick2uniq[5003]['P'], "easyccg")
k = ConjEntail(P)

Subj: A BIG CAT
ConjVP: BE OPEN A PLASTIC DRAWER WITH ITS PAW AND BE JUMP INSIDE


building tree 71...


In [30]:
k[0].wholeStr

'A BIG CAT BE OPEN A PLASTIC DRAWER WITH ITS PAW'

In [31]:
vars(k[0])

{'parent': None,
 'children': [nt: NP ((e,t),t) fa 1 None False,
  nt: S[dcl]\NP (((e,t),t),t) fa 2 None False],
 'sisters': [],
 'depth': 0,
 'cat': S[dcl],
 'ruleType': 'ba',
 'wholeStr': 'A BIG CAT BE OPEN A PLASTIC DRAWER WITH ITS PAW',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x111b7bb50>,
 'impSign': None,
 'note': None,
 'number': None}

In [34]:
BE = k[0].children[1]
vars(BE)

{'parent': nt: S[dcl]\NP (((e,t),t),t) ba 1 None False,
 'children': [lf: (S[dcl]\NP)/(S[ng]\NP) ((((e,t),t),t),(((e,t),t),t)) be VBZ 3 mono:None imp:None True,
  nt: S[ng]\NP (((e,t),t),t) ba 3 None False],
 'sisters': [nt: (S[dcl]\NP)\(S[dcl]\NP) ((((e,t),t),t),(((e,t),t),t)) conj 2 None False],
 'depth': 2,
 'cat': S[dcl]\NP,
 'ruleType': 'fa',
 'wholeStr': 'BE OPEN A PLASTIC DRAWER WITH ITS PAW',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x1119a6430>,
 'impSign': None,
 'note': None,
 'number': None}

In [35]:
OPEN = BE.children[1]
vars(OPEN)

{'parent': nt: S[dcl]\NP (((e,t),t),t) fa 2 None False,
 'children': [nt: S[ng]\NP (((e,t),t),t) fa 4 None False,
  nt: (S\NP)\(S\NP) ((((e,t),t),t),(((e,t),t),t)) fa 4 None False],
 'sisters': [lf: (S[dcl]\NP)/(S[ng]\NP) ((((e,t),t),t),(((e,t),t),t)) be VBZ 3 mono:None imp:None True],
 'depth': 3,
 'cat': S[ng]\NP,
 'ruleType': 'ba',
 'wholeStr': 'OPEN A PLASTIC DRAWER WITH ITS PAW',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x111b67c10>,
 'impSign': None,
 'note': None,
 'number': None}

In [38]:
OBJ = OPEN.children[0]
OBJ = OBJ.children[1]
vars(OBJ)

{'parent': nt: S[ng]\NP (((e,t),t),t) fa 4 None False,
 'children': [lf: NP/N ((e,t),((e,t),t)) a DT 6 mono:None imp:None True,
  nt: N (e,t) fa 6 None False],
 'sisters': [lf: (S[ng]\NP)/NP (((e,t),t),(((e,t),t),t)) open VBG 5 mono:None imp:None True],
 'depth': 5,
 'cat': NP,
 'ruleType': 'fa',
 'wholeStr': 'A PLASTIC DRAWER',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x111b6ca30>,
 'impSign': None,
 'note': None,
 'number': None}

In [45]:
N = OBJ.children[1]
vars(N.children[0])

{'parent': nt: N (e,t) fa 6 None False,
 'children': [],
 'sisters': [lf: N (e,t) drawer NN 7 mono:None imp:None True],
 'depth': 7,
 'cat': N/N,
 'chunk': 'XX',
 'entity': 'O',
 'lemma': 'plastic',
 'pos': 'JJ',
 'span': 1,
 'start': 6,
 'word': 'plastic',
 'wholeStr': 'PLASTIC',
 'word_raw': 'plastic',
 'visited': True,
 'span_id': None,
 'impType': <getMono.ImpType at 0x1119e8340>,
 'impSign': None,
 'fixed': False,
 'note': None,
 'number': None}

In [44]:
vars(N.cat)

{'direction': 's',
 'left': None,
 'right': None,
 'originalType': 'N',
 'typeWOpolarity': 'N',
 'typeWOfeats': 'N',
 'monotonicity': None,
 'lex_polarity': None,
 'word': None,
 'semCat': (e,t),
 'regexBrk': '\\[[^\\[\\]]+?\\]'}

In [47]:
[]+["hello"]

['hello']

### Extracting NPs

In [113]:
def ExtractNPs(P):    
    if P.cat.originalType == 'NP':
        return [P]
    elif len(P.children) == 0:
        return []
    else:
        tmp = []
        for c in P.children:
            tmp += ExtractNPs(c)
        return tmp

In [52]:
vars(P)

{'leafNodes': [lf: NP/N ((e,t),((e,t),t)) a DT 2 mono:None imp:None True,
  lf: N/N ((e,t),(e,t)) big JJ 3 mono:None imp:None True,
  lf: N (e,t) cat NN 3 mono:None imp:None True,
  lf: (S[dcl]\NP)/(S[ng]\NP) ((((e,t),t),t),(((e,t),t),t)) be VBZ 3 mono:None imp:None True,
  lf: (S[ng]\NP)/NP (((e,t),t),(((e,t),t),t)) open VBG 5 mono:None imp:None True,
  lf: NP/N ((e,t),((e,t),t)) a DT 6 mono:None imp:None True,
  lf: N/N ((e,t),(e,t)) plastic JJ 7 mono:None imp:None True,
  lf: N (e,t) drawer NN 7 mono:None imp:None True,
  lf: ((S\NP)\(S\NP))/NP (((e,t),t),((((e,t),t),t),(((e,t),t),t))) with IN 5 mono:None imp:None True,
  lf: NP/(N/PP) ((pp,(e,t)),((e,t),t)) its PRP$ 6 mono:None imp:None True,
  lf: N/PP (pp,(e,t)) paw NNS 6 mono:None imp:None True,
  lf: conj CONJ and CC 3 mono:None imp:None True,
  lf: (S[dcl]\NP)/(S[ng]\NP) ((((e,t),t),t),(((e,t),t),t)) be VBZ 4 mono:None imp:None True,
  lf: S[ng]\NP (((e,t),t),t) jump VBG 5 mono:None imp:None True,
  lf: (S\NP)\(S\NP) ((((e,t),

In [54]:
NPs = ExtractNPs(P.root)

In [55]:
for NP in NPs:
    print(NP.wholeStr)

A BIG CAT
A PLASTIC DRAWER
ITS PAW


#### Testing ExtractNPs

In [57]:
for i in CONJ_ID:
    P = trees.build_one_tree(sick2uniq[i]['P'], "easyccg")
    NPs = ExtractNPs(P.root)
    print(P.wholeStr)
    for NP in NPs:
        print(NP.wholeStr)
    print()

A BIG CAT BE OPEN A PLASTIC DRAWER WITH ITS PAW AND BE JUMP INSIDE
A BIG CAT
A PLASTIC DRAWER
ITS PAW

AN ADULT BE IN A AMPHITHEATER AND BE TALK TO A BOY
AN ADULT
A AMPHITHEATER
A BOY

A BLACK DOG BE RUN IN A GRASS AND ITS TONGUE BE HANG OUT
A BLACK DOG
A GRASS
ITS TONGUE

A MAN BE SMOKE A CIGARETTE IN A STRAW HAT
A MAN
A CIGARETTE
A STRAW HAT

A ICE HOCKEY GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL
A ICE HOCKEY GOALKEEPER
A YELLOW JERSEY
A GOAL

AN ELD MAN BE SIT ON A BENCH AND BE WEAR A GRAY JACKET AND BLACK PANT
AN ELD MAN
A BENCH
A GRAY JACKET AND BLACK PANT

A MAN BE SMOKE A CIGARETTE IN A STRAW HAT
A MAN
A CIGARETTE
A STRAW HAT

A MAN IN A BLACK JERSEY BE STAND IN A GYM
A MAN IN A BLACK JERSEY
A GYM



building tree 71...
building tree 3315...
building tree 167...
building tree 549...
building tree 4304...
building tree 3344...
building tree 549...
building tree 1413...


### Extracting adj

In [59]:
P.wholeStr

'A MAN IN A BLACK JERSEY BE STAND IN A GYM'

In [64]:
vars(NPs[0])

{'parent': None,
 'children': [lf: NP/N ((e,t),((e,t),t)) a DT 2 mono:None imp:None True,
  nt: N (e,t) ba 2 None False],
 'sisters': [],
 'depth': 1,
 'cat': NP,
 'ruleType': 'fa',
 'wholeStr': 'A MAN IN A BLACK JERSEY',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x111c30130>,
 'impSign': None,
 'note': None,
 'number': None}

In [105]:
def ExtractAdjs(node):
    # print(node.wholeStr, node.children)
    if len(node.children) == 0:
        # print(node.pos, node.cat.originalType)
        if node.pos == "JJ":
            return [node]
        elif node.pos == "NN" and not (node.cat.originalType == 'N'):
            return [node]
        else: return []
    else:
        tmp = []
        for c in node.children:
            # print(c)
            tmp += ExtractAdjs(c)
        return tmp

In [106]:
for i in CONJ_ID:
    P = trees.build_one_tree(sick2uniq[i]['P'], "easyccg")
    NPs = ExtractNPs(P.root)
    
    for NP in NPs:
        Adjs = ExtractAdjs(NP)
        print(NP.wholeStr)
        for Adj in Adjs:
            print(Adj.wholeStr)
        print()

A BIG CAT
BIG

A PLASTIC DRAWER
PLASTIC

ITS PAW

AN ADULT

A AMPHITHEATER

A BOY

A BLACK DOG
BLACK

A GRASS

ITS TONGUE
TONGUE

A MAN

A CIGARETTE

A STRAW HAT
STRAW

A ICE HOCKEY GOALKEEPER
ICE
HOCKEY

A YELLOW JERSEY
YELLOW

A GOAL

AN ELD MAN

A BENCH

A GRAY JACKET AND BLACK PANT
GRAY
BLACK

A MAN

A CIGARETTE

A STRAW HAT
STRAW

A MAN IN A BLACK JERSEY
BLACK

A GYM



building tree 71...
building tree 3315...
building tree 167...
building tree 549...
building tree 4304...
building tree 3344...
building tree 549...
building tree 1413...


##### finding out a way to identify modifiers in noun compounds

In [71]:
P = trees.build_one_tree(sick2uniq[6544]['P'], "easyccg")

building tree 4304...


In [75]:
P.root.children

[nt: NP ((e,t),t) fa 1 None False, nt: S[dcl]\NP (((e,t),t),t) ba 1 None False]

In [83]:
Subj = P.root.children[0]

In [77]:
Subj.wholeStr

'A ICE HOCKEY GOALKEEPER'

In [84]:
Subj = Subj.children[1]

In [90]:
vars(Subj.children[1].cat)

{'direction': 's',
 'left': None,
 'right': None,
 'originalType': 'N',
 'typeWOpolarity': 'N',
 'typeWOfeats': 'N',
 'monotonicity': None,
 'lex_polarity': None,
 'word': 'goalkeeper',
 'semCat': (e,t),
 'regexBrk': '\\[[^\\[\\]]+?\\]'}

In [91]:
vars(Subj.children[0].children[0].cat)

{'direction': 'r',
 'left': N/N,
 'right': N/N,
 'originalType': '(N/N)/(N/N)',
 'typeWOpolarity': '(N/N)/(N/N)',
 'typeWOfeats': '(N/N)/(N/N)',
 'monotonicity': None,
 'lex_polarity': None,
 'word': 'ice',
 'semCat': (((e,t),(e,t)),((e,t),(e,t))),
 'regexBrk': '\\[[^\\[\\]]+?\\]'}

In [104]:
ExtractAdjs(Subj)

ICE HOCKEY GOALKEEPER [nt: N/N ((e,t),(e,t)) fa 3 None False, lf: N (e,t) goalkeeper NN 3 mono:None imp:None True]
nt: N/N ((e,t),(e,t)) fa 3 None False
ICE HOCKEY [lf: (N/N)/(N/N) (((e,t),(e,t)),((e,t),(e,t))) ice NN 4 mono:None imp:None True, lf: N/N ((e,t),(e,t)) hockey NN 4 mono:None imp:None True]
lf: (N/N)/(N/N) (((e,t),(e,t)),((e,t),(e,t))) ice NN 4 mono:None imp:None True
ICE []
NN (N/N)/(N/N)
lf: N/N ((e,t),(e,t)) hockey NN 4 mono:None imp:None True
HOCKEY []
NN N/N
lf: N (e,t) goalkeeper NN 3 mono:None imp:None True
GOALKEEPER []
NN N


[lf: (N/N)/(N/N) (((e,t),(e,t)),((e,t),(e,t))) ice NN 4 mono:None imp:None True,
 lf: N/N ((e,t),(e,t)) hockey NN 4 mono:None imp:None True]

### Removing one Adjective

In [107]:
P = trees.build_one_tree(sick2uniq[6544]['P'], "easyccg")

building tree 4304...


In [114]:
NPs = ExtractNPs(P.root)
NPs

[nt: NP ((e,t),t) fa 1 None False,
 nt: NP ((e,t),t) fa 4 None False,
 nt: NP ((e,t),t) fa 5 None False]

In [117]:
vars(NPs[0])

{'parent': nt: S[dcl] t ba 0 None False,
 'children': [lf: NP/N ((e,t),((e,t),t)) a DT 2 mono:None imp:None True,
  nt: N (e,t) fa 2 None False],
 'sisters': [nt: S[dcl]\NP (((e,t),t),t) ba 1 None False],
 'depth': 1,
 'cat': NP,
 'ruleType': 'fa',
 'wholeStr': 'A ICE HOCKEY GOALKEEPER',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x112d53f40>,
 'impSign': None,
 'note': None,
 'number': None}

In [116]:
P.wholeStr

'A ICE HOCKEY GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL'

In [120]:
P1 = P.root.copy()
vars(P1)

{'parent': None,
 'children': [nt: NP ((e,t),t) fa 1 None False,
  nt: S[dcl]\NP (((e,t),t),t) ba 1 None False],
 'sisters': [],
 'depth': 0,
 'cat': S[dcl],
 'ruleType': 'ba',
 'wholeStr': 'A ICE HOCKEY GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x112dbc2e0>,
 'impSign': None,
 'note': None,
 'number': None}

In [163]:
def findAdj(root, adj):
    if len(root.children) == 0:
        return None
    else:
        root.children[0].parent = root
        root.children[1].parent = root
        root.children[0].sisters = root.children[1]
        root.children[1].sisters = root.children[0]
        
        if adj.wholeStr == root.children[0].wholeStr and adj.parent.wholeStr == root.children[0].parent.wholeStr:
            return root.children[0]
        elif adj.wholeStr == root.children[1].wholeStr and adj.parent.wholeStr == root.children[1].parent.wholeStr:
            return root.children[1]
        else:
            res = findAdj(root.children[0], adj)
            if res == None:
                res = findAdj(root.children[1], adj)
            return res

def removeAdj(P,NP,adj):
    root = P.root.copy()
    adj = findAdj(root, adj)
    
    #     par.par                par.par
    #       /     \                 /     \ 
    #     par    rest    ->       sister   rest
    #    /   \
    #  adj   sister
    parpar = adj.parent.parent
    sister = adj.sisters
    
    sister.sisters = None
    sister.parent = parpar
    
    for i in range(2):
        if parpar.children[i] == adj.parent:
            parpar.children[i] = sister
            
    # updating wholeStr
    parent = sister.parent
    while parent:
        parent.assignWholeStr()
        parent = parent.parent
    
    return root

def RemoveOneAdj(P, NP):
    # for each adj in NP, remove adj and store the sentence in tmp
    tmp = []
    ADJs = ExtractAdjs(NP)
    
    for adj in ADJs:
        # remove adj from NP
        tmp.append(removeAdj(P,NP,adj))
        
    return tmp
        

In [123]:
NPs

[nt: NP ((e,t),t) fa 1 None False,
 nt: NP ((e,t),t) fa 4 None False,
 nt: NP ((e,t),t) fa 5 None False]

In [124]:
ADJs = ExtractAdjs(NPs[0])
ADJs

[lf: (N/N)/(N/N) (((e,t),(e,t)),((e,t),(e,t))) ice NN 4 mono:None imp:None True,
 lf: N/N ((e,t),(e,t)) hockey NN 4 mono:None imp:None True]

In [150]:
root = P.root.copy()
adj = findAdj(root,ADJs[0])
adj

lf: (N/N)/(N/N) (((e,t),(e,t)),((e,t),(e,t))) ice NN 4 mono:None imp:None True

In [151]:
parpar = adj.parent.parent
sister = adj.sisters

In [153]:
adj.parent in parpar.children

True

In [155]:
print(root.parent)

None


In [161]:
removed = removeAdj(P, NPs[0],ADJs[2])

IndexError: list index out of range

In [160]:
vars(removed)

{'parent': None,
 'children': [nt: NP ((e,t),t) fa 1 None False,
  nt: S[dcl]\NP (((e,t),t),t) ba 1 None False],
 'sisters': [],
 'depth': 0,
 'cat': S[dcl],
 'ruleType': 'ba',
 'wholeStr': 'A ICE GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL',
 'visited': False,
 'span_id': None,
 'impType': <getMono.ImpType at 0x112f08610>,
 'impSign': None,
 'note': None,
 'number': None}

#### Testing Remove One Adjective

In [165]:
for i in CONJ_ID:
    P = trees.build_one_tree(sick2uniq[i]['P'], "easyccg")
    NPs = ExtractNPs(P.root)
    print(P.wholeStr)
    
    for NP in NPs:
        removed = RemoveOneAdj(P, NP)
        print(NP.wholeStr)
        for r in removed:
            print(r.wholeStr)
        print()
        

A BIG CAT BE OPEN A PLASTIC DRAWER WITH ITS PAW AND BE JUMP INSIDE
A BIG CAT
A CAT BE OPEN A PLASTIC DRAWER WITH ITS PAW AND BE JUMP INSIDE

A PLASTIC DRAWER
A BIG CAT BE OPEN A DRAWER WITH ITS PAW AND BE JUMP INSIDE

ITS PAW

AN ADULT BE IN A AMPHITHEATER AND BE TALK TO A BOY
AN ADULT

A AMPHITHEATER

A BOY

A BLACK DOG BE RUN IN A GRASS AND ITS TONGUE BE HANG OUT
A BLACK DOG
A DOG BE RUN IN A GRASS AND ITS TONGUE BE HANG OUT

A GRASS

ITS TONGUE
A BLACK DOG BE RUN IN A GRASS AND ITS BE HANG OUT

A MAN BE SMOKE A CIGARETTE IN A STRAW HAT
A MAN

A CIGARETTE

A STRAW HAT
A MAN BE SMOKE A CIGARETTE IN A HAT

A ICE HOCKEY GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL
A ICE HOCKEY GOALKEEPER
A HOCKEY GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL
A ICE GOALKEEPER BE WEAR A YELLOW JERSEY AND BE DEFEND A GOAL

A YELLOW JERSEY
A ICE HOCKEY GOALKEEPER BE WEAR A JERSEY AND BE DEFEND A GOAL

A GOAL

AN ELD MAN BE SIT ON A BENCH AND BE WEAR A GRAY JACKET AND BLACK PANT
AN ELD M

building tree 71...
building tree 3315...
building tree 167...
building tree 549...
building tree 4304...
building tree 3344...
building tree 549...
building tree 1413...
